In [1]:
import pandas as pd
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
from scipy import stats
import pystan as stan
import arviz

# Set default figure size
plt.rcParams["figure.figsize"] = (15, 10)
pd.plotting.register_matplotlib_converters()

In [2]:
# Here"s my minute data for the S&P 500
spx_minute = pd.read_csv("SPX_1min.csv", header=0,names=["datetime", "open", "high", "low", "close"],
                                  index_col="datetime", parse_dates=True)

In [3]:
# Here"s the function for calculating the 1-min RV, as discussed in my last post
def rv_calc(data):
    results = {}
    
    for idx, data in data.groupby(data.index.date):
        returns = np.log(data["close"]) - np.log(data["close"].shift(1))
        results[idx] = np.sum(returns**2)
        
    return pd.Series(results)

In [4]:
spx_rvar = rv_calc(spx_minute)

In [5]:
conn = sqlite3.Connection("data.db")
spx_data = pd.read_sql("SELECT * FROM prices WHERE ticker='^GSPC'", conn, index_col="date", parse_dates="date")
spx_returns = np.log(spx_data["close"]) - np.log(spx_data["close"].shift(1))
spx_returns = spx_returns.dropna()

vix_data = pd.read_sql("SELECT * FROM prices WHERE ticker='^VIX'", conn, index_col="date", parse_dates="date")
# This puts it into units of daily standard deviation
vix = vix_data["close"] / np.sqrt(252) / 100

In [6]:
def create_lags(series, lags, name="x"):
    """
    Creates a dataframe with lagged values of the given series.
    Generates columns named x_{n} which means the value of each row is the value of the original series lagged n times
    """
    result = pd.DataFrame(index=series.index)
    result[f"{name}_t"] = series
    
    for n in range(lags):
        result[f"{name}_t-{n+1}"] = series.shift((n+1))
        
    return result

In [202]:
y = pd.DataFrame(index=spx_rvar.index, columns=["t+1", "t+2", "t+3", "t+4", "t+5"], dtype="float64")
N = 5

for x in range(len(spx_rvar) - 5):
    y.iloc[x] = spx_rvar.iloc[x+1:x+6]

In [205]:
vix_lags = create_lags(vix, 21, name="vix")
return_lags = create_lags(spx_returns, 21, name="returns")
rv_lags = create_lags(np.sqrt(spx_rvar), 21, name="rv")

x = pd.concat([vix_lags, return_lags, rv_lags], axis=1).dropna()
y = np.sqrt(y).dropna()

common_index = x.index.intersection(y.index)
x = x.loc[common_index]
y = y.loc[common_index]

In [219]:
import torch
import torch.nn as nn
from torch.distributions import Categorical, Normal, Independent, MixtureSameFamily
from torch.optim.swa_utils import AveragedModel, SWALR

torch.set_default_dtype(torch.float64)

In [220]:
class MDN(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_dim, n_components):
        super().__init__()
        self.n_components = n_components
        # Last layer output dimension rationale:
        # Need two parameters for each distributionm thus 2 * n_components.
        # Need each of those for each output dimension, thus that multiplication
        self.norm_network = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, 2 * n_components * out_dim)
        )
        self.cat_network = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, n_components)
        )
        
    def forward(self, x):
        norm_params = self.norm_network(x)
        mean, std = torch.split(norm_params, norm_params.shape[1] // 2, dim=1)
        normal = Normal(mean, torch.exp(std))
        
        cat_params = self.cat_network(x)
        cat = Categorical(logits=cat_params)
        
        return MixtureSameFamily(cat, normal)

In [221]:
test_index = int(len(x) * .75)
train_x = torch.Tensor(x.iloc[:test_index].values)
train_y = torch.Tensor(y.iloc[:test_index].values)
test_x = torch.Tensor(x.iloc[test_index:].values)
test_y = torch.Tensor(y.iloc[test_index:].values)

in_dim = len(x.columns)
out_dim = 5
n_components = 10
hidden_dim = int(np.mean([in_dim, 2 * n_components * out_dim]))

In [222]:
model = MDN(in_dim, out_dim, hidden_dim, n_components)
optimizer = torch.optim.AdamW(model.parameters(), lr=.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 100, 2)

swa_model = AveragedModel(model)
swa_start = 750
swa_scheduler = SWALR(optimizer, swa_lr=0.001, anneal_epochs=10, anneal_strategy="cos")

train_losses = []
validation_losses = []
for epoch in range(1000):
    optimizer.zero_grad()
    output = model(train_x)

    train_loss = -output.log_prob(train_y).sum()
    train_losses.append(train_loss.detach())
    
    test_loss = -model(test_x).log_prob(test_y).sum()
    validation_losses.append(test_loss.detach())
    
    train_loss.backward()
    optimizer.step()
    
    if epoch > swa_start:
        swa_model.update_parameters(model)
        swa_scheduler.step()
    else:
        scheduler.step()

ValueError: `mixture_distribution component` (10) does not equal `component_distribution.batch_shape[-1]` (50)